In [1]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer
import torch
from torch.nn import functional as F 
model = XLMRobertaForTokenClassification.from_pretrained('../output/model/custom_tokenizer/checkpoint-75')
tokenizer = XLMRobertaTokenizer.from_pretrained('../output/model/custom_tokenizer/checkpoint-75')

/Users/waterondaway/Laboratory/medicine-scan-lm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def calling_model(text):
    DRUG_NAME = ""
    DOSAGE = ""
    FORM = ""
    DRUG_REG_NO = ""
    MFG_DATE = ""
    EXP_DATE = ""
    WARNINGS = ""
    INDICATIONS = ""
    USAGE_INSTRUCTIONS = ""
    OBJECT = ""
    label2id = {
        "O": 0,
        "B-DRUG_NAME": 1, "I-DRUG_NAME": 2,
        "B-DOSAGE": 3, "I-DOSAGE": 4,
        "B-FORM": 5, "I-FORM": 6,
        "B-DRUG_REG_NO": 7, "I-DRUG_REG_NO": 8,
        "B-MFG_DATE": 9, "I-MFG_DATE": 10,
        "B-EXP_DATE": 11, "I-EXP_DATE": 12,
        "B-WARNINGS": 13, "I-WARNINGS": 14,
        "B-INDICATIONS": 15, "I-INDICATIONS": 16,
        "B-USAGE_INSTRUCTIONS": 17, "I-USAGE_INSTRUCTIONS": 18
    }
    id2label = {v: k for k, v in label2id.items()}
    tokens = tokenizer.tokenize(text)

    print(tokens)

    tokenized_input = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokenized_input)
    logits = outputs.logits
    print(logits)
    probs = F.softmax(logits, dim=-1)
    predictions = torch.argmax(probs, dim=-1)
    for i, token in enumerate(tokens):
        pred_label = id2label[predictions[0][i].item()]
        confidence = probs[0][i][predictions[0][i]].item()*100
        print(token, pred_label, confidence)
        if(pred_label == "O"):
            pred_label == "O"
        else:
            pred_label = pred_label.split('-')[1]
        if(pred_label == "DRUG_NAME"):
            DRUG_NAME += token
        elif(pred_label == "FORM"):
            FORM += token
        elif(pred_label == "DOSAGE"):
            DOSAGE += token
        elif(pred_label == "DRUG_REG_NO"):
            DRUG_REG_NO += token
        elif(pred_label == "MFG_DATE"):
            MFG_DATE += token
        elif(pred_label == "EXP_DATE"):
            EXP_DATE += token
        elif(pred_label == "WARNINGS"):
            WARNINGS += token
        elif(pred_label == "INDICATIONS"):
            INDICATIONS += token
        elif(pred_label == "USAGE_INSTRUCTIONS"):
            USAGE_INSTRUCTIONS += token
        elif(pred_label == "O"):
            OBJECT += token

In [ ]:
def calling_model(text):
    DRUG_NAME = ""
    OBJECT = ""
    label2id = {
        "O": 0,
        "B-DRUG_NAME": 1, "I-DRUG_NAME": 2,
    }
    id2label = {v: k for k, v in label2id.items()}
    tokens = tokenizer.tokenize(text)

    print(tokens)

    tokenized_input = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokenized_input)
    logits = outputs.logits
    print(logits)
    probs = F.softmax(logits, dim=-1)
    predictions = torch.argmax(probs, dim=-1)
    for i, token in enumerate(tokens):
        pred_label = id2label[predictions[0][i].item()]
        confidence = probs[0][i][predictions[0][i]].item()*100
        print(token, pred_label, confidence)
        if(pred_label == "O"):
            pred_label == "O"
        else:
            pred_label = pred_label.split('-')[1]
        if(pred_label == "DRUG_NAME"):
            DRUG_NAME += token
        elif(pred_label == "FORM"):
            FORM += token
        elif(pred_label == "DOSAGE"):
            DOSAGE += token
        elif(pred_label == "DRUG_REG_NO"):
            DRUG_REG_NO += token
        elif(pred_label == "MFG_DATE"):
            MFG_DATE += token
        elif(pred_label == "EXP_DATE"):
            EXP_DATE += token
        elif(pred_label == "WARNINGS"):
            WARNINGS += token
        elif(pred_label == "INDICATIONS"):
            INDICATIONS += token
        elif(pred_label == "USAGE_INSTRUCTIONS"):
            USAGE_INSTRUCTIONS += token
        elif(pred_label == "O"):
            OBJECT += token

In [10]:
calling_model("ไอบูโพรเฟน")

['ไอบูโพรเฟน']
tensor([[[ 1.9579e+00, -2.8307e-01, -1.2438e+00],
         [ 3.2329e+00, -2.9579e-02, -3.0413e+00],
         [ 4.3937e+00, -3.1662e-03, -4.8254e+00]]])
ไอบูโพรเฟน O 87.18053698539734


In [11]:
calling_model("ไดอะซีแพม")

['ไดอะซีแพม']
tensor([[[ 1.9508e+00, -2.8459e-01, -1.2348e+00],
         [ 3.2455e+00, -2.9956e-02, -3.0624e+00],
         [ 4.3922e+00, -1.0591e-03, -4.8349e+00]]])
ไดอะซีแพม O 87.08502650260925
